In [ ]:
import time, sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

role = sagemaker.get_execution_role()

# 1) 방금 fit한 estimator에서 model_data(S3 model.tar.gz) 얻기
model_data_s3 = estimator.model_data
print("model_data:", model_data_s3)  # s3://.../model.tar.gz 이어야 함

# 2) 실시간용 스크립트로 모델 정의 (model_fn/input_fn/predict_fn/output_fn 있는 파일)
rt_model = PyTorchModel(
    model_data=model_data_s3,
    role=role,
    entry_point="realtime_infer.py",   # ★ 엔드포인트용
    source_dir=".",                    # realtime_infer.py + requirements.txt 위치
    framework_version="2.0.0",
    py_version="py310",
)

# 3) 배포
endpoint_name = f"say1-5team-skin-endpoint-{time.strftime('%Y-%m-%d-%H-%M-%S')}"
predictor = rt_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",    # YOLO면 GPU 권장
    endpoint_name=endpoint_name,
    tags=[{"Key":"project","Value":"pre-5team"}],
)

# 4) 호출 설정 (바이너리 JPEG -> JSON 응답)
predictor.serializer = IdentitySerializer("image/jpeg")
predictor.deserializer = JSONDeserializer()

print("✅ Endpoint:", predictor.endpoint_name)
